# Tutorial

We now illustrate the basic capabilities of the ``respy`` package in a simple tutorial. The tutorial covers the following topics:

1. [How to specify a model - parameters and options.](#How-to-specify-a-model---parameters-and-options.)
2. [How to solve a model.](#How-to-solve-a-model.)
3. [How to simulate a data set.](#How-to-simulate-a-data-set.)
4. [How to estimate model parameters.](#How-to-estimate-model-parameters.)

After the [installation](installation.rst), the convention is to import ``respy`` as follows to get access to all exposed functions.

In [1]:
import respy as rp

## How to specify a model - parameters and options.

In order to define a model, ``respy`` relies on two objects - parameters and options. The difference between the two is that parameters are affected by the optimization whereas options stay the same. Here, only the following parameters and options are explained. For a complete overview on parameters and options, visit the section [model specification](../software/model-specification.rst).

As an example, we turn to the first model of Keane and Wolpin (1994). The following function returns the parameters and options.

In [2]:
params, options = rp.get_example_model("kw_data_one")

As said before, options are everything not affected by the optimization. The value under ``"n_periods"`` defines the number of periods in the model meaning the available time frame in which individual can make their decisions.

Choices which are available to individuals within the model are specified under ``"choices"``. An empty dictionary as a value signals that this choice relies on default values generated inside of ``respy`` or does not need information at all. Under ``"edu"`` we can see which options are available for each choice.

- ``"max"`` defines the maximum amount of experience which can be accumulated by individuals using this choice.
- ``"start"`` is a list and contains different initial experience levels of this choice. Here, individuals start with 10 years of experience.
- ``"lagged"`` defines the share of individual with this choice and initial experience level having ``"edu"`` as the initial lagged choice when entering the model.
- ``"share"`` influences the probability of being a particular type.

The rest of the options includes seeds, draws to simulate the flow utilities in each period and other model parameters. Furthermore, there is a lot of logic hidden from the user to offer a comprehensible starting point. For a deeper understanding of the mechanics, the user is refered to the section [model specfication](../software/model-specification.rst).

In [3]:
options

{'choices': {'a': {},
  'b': {},
  'edu': {'max': 20, 'start': [10], 'lagged': [1], 'share': [1]},
  'home': {}},
 'estimation_draws': 200,
 'estimation_seed': 500,
 'estimation_tau': 500,
 'interpolation_points': -1,
 'n_periods': 40,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'solution_draws': 500,
 'solution_seed': 456}

The parameter specification includes all parameters of the model which are affected by the optimization routine.

In [4]:
params

value  fixed  lower  upper  \
category    name                                                        
delta       delta                          0.950  False    0.7    1.0   
wage_a      constant                       9.210  False    NaN    NaN   
            exp_edu                        0.038  False    NaN    NaN   
            exp_a                          0.033  False    NaN    NaN   
            exp_a_square                  -0.050  False    NaN    NaN   
            exp_b                          0.000  False    NaN    NaN   
            exp_b_square                   0.000  False    NaN    NaN   
            hs_graduate                    0.000  False    NaN    NaN   
            co_graduate                    0.000  False    NaN    NaN   
            period                         0.000  False    NaN    NaN   
            is_minor                       0.000  False    NaN    NaN   
            any_exp_a                      0.000  False    NaN    NaN   
            work_a_lagged                  0.000  False    NaN    NaN   
nonpec_a    constant                       0.000  False    NaN    NaN   
            not_exp_a_lagged               0.000  False    NaN    NaN   
            not_any_exp_a                  0.000  False    NaN    NaN   
            hs_graduate                    0.000  False    NaN    NaN   
            co_graduate                    0.000  False    NaN    NaN   
wage_b      constant                       8.480  False    NaN    NaN   
            exp_edu                        0.070  False    NaN    NaN   
            exp_a                          0.022  False    NaN    NaN   
            exp_a_square                  -0.050  False    NaN    NaN   
            exp_b                          0.067  False    NaN    NaN   
            exp_b_square                  -0.100  False    NaN    NaN   
            hs_graduate                    0.000  False    NaN    NaN   
            co_graduate                    0.000  False    NaN    NaN   
            period                         0.000  False    NaN    NaN   
            is_minor                       0.000  False    NaN    NaN   
            any_exp_b                      0.000  False    NaN    NaN   
            work_b_lagged                  0.000  False    NaN    NaN   
nonpec_b    constant                       0.000  False    NaN    NaN   
            not_exp_b_lagged               0.000  False    NaN    NaN   
            not_any_exp_b                  0.000  False    NaN    NaN   
            hs_graduate                    0.000  False    NaN    NaN   
            co_graduate                    0.000  False    NaN    NaN   
nonpec_edu  constant                       0.000  False    NaN    NaN   
            is_return_not_high_school  -4000.000  False    NaN    NaN   
            is_return_high_school      -4000.000  False    NaN    NaN   
            period                         0.000  False    NaN    NaN   
            is_minor                       0.000  False    NaN    NaN   
            hs_graduate                    0.000  False    NaN    NaN   
            co_graduate                    0.000  False    NaN    NaN   
nonpec_home constant                   17750.000  False    NaN    NaN   
            is_young_adult                 0.000  False    NaN    NaN   
            is_adult                       0.000  False    NaN    NaN   
            hs_graduate                    0.000  False    NaN    NaN   
            co_graduate                    0.000  False    NaN    NaN   
shocks      sd_a                           0.200  False    NaN    NaN   
            sd_b                           0.250  False    NaN    NaN   
            sd_edu                      1500.000  False    NaN    NaN   
            sd_home                     1500.000  False    NaN    NaN   
            corr_a_b                       0.000  False    NaN    NaN   
            corr_edu_a                     0.000  False    NaN    NaN   
            corr_edu_b      

## How to solve a model.

The solution of the model is the value of each state within the model assuming optimal decision in this and future periods. In ``respy``, the solution is represented with the :class:`~respy.state_space.StateSpace`, a class including the model solution. The solution combines the following steps:

- Building the state space.
- Solving the model by finding optimal decision in each state via backward induction.

The ``state_space`` is normally hidden from the user and only an internal construct. For a better understanding of how modeling works, the state space is exposed to the user.

In [5]:
state_space = rp.solve(params, options)

## How to simulate a data set.

Simulating a data set is as easy as passing model inputs to :func:`~respy.simulate.simulate`. The return values are the ``state_space`` of the model and ``df``, a data set of simulated agents as a :class:`pandas.DataFrame`. The number of simulated agents can be set in the options under ``"simulation_agents"``.

In [6]:
state_space, df = rp.simulate(params, options)

## How to estimate model parameters.

To estimate model parameters via maximum likelihood, ``respy`` relies on [``estimagic``](https://github.com/OpenSourceEconomics/estimagic), an open-source tool to estimate structural models and more. First, we need the criterion function of the model which returns the likelihood value of the data given the model.

In [7]:
crit_func = rp.get_crit_func(params, options, df)

The criterion function takes the parameters as an input and returns the likelihood value.

In [8]:
crit_val = crit_func(params)
crit_val

/mnt/c/Users/tobia/git/respy/respy/conditional_draws.py:79: RuntimeWarning: overflow encountered in exp
  draws[:, :2] = np.clip(np.exp(draws[:, :2]), 0.0, HUGE_FLOAT)


-333.31969190264334

To estimate the model parameters, import the :func:`~estimagic.optimization.optimize.minimize` from ``estimagic``.

In [9]:
import numpy as np
from estimagic.optimization.optimize import minimize

For ``estimagic``, we need to pass constraints on the parameters in a list containing dictionaries. Each dictionary is a constraint. A constraint includes two components: First, we need to tell ``estimagic`` which paramters we want to constrain. This is achieved by a ``pd.query`` on the index of the frame. Then, define the type of the constraint. Here, we only impose a constraint on the sd-correlation matrix of the constraints.

In [10]:
constr = [{"query": "category == 'shocks'", "type": "sdcorr"}]

Furthermore, we add a column ``"group"`` which is identical to the category column, so that we can observe parameters in groups.

In [11]:
params["group"] = params.reset_index().category.values
params.lower = params.value

In [12]:
mnimize(crit_func, params, "pygmo_bee_colony", algo_options={"popsize": 2}, db_options={"rollover": 200}, constraints=constr, dashboard=True)

/mnt/c/Users/tobia/git/respy/respy/conditional_draws.py:79: RuntimeWarning: overflow encountered in exp
  draws[:, :2] = np.clip(np.exp(draws[:, :2]), 0.0, HUGE_FLOAT)


Bokeh app running at: http://localhost:5039/


/mnt/c/Users/tobia/git/respy/respy/conditional_draws.py:79: RuntimeWarning: overflow encountered in exp
  draws[:, :2] = np.clip(np.exp(draws[:, :2]), 0.0, HUGE_FLOAT)
/mnt/c/Users/tobia/git/respy/respy/likelihood.py:292: RuntimeWarning: divide by zero encountered in log
  contribs = np.clip(np.log(contribs), -HUGE_FLOAT, HUGE_FLOAT)


KeyboardInterrupt: 

In [ ]:
%debug

## References

> Keane, M. P. and  Wolpin, K. I. (1994b). [The Solution and Estimation of Discrete Choice Dynamic Programming Models by Simulation and Interpolation: Monte Carlo Evidence](https://www.minneapolisfed.org/research/staff-reports/the-solution-and-estimation-of-discrete-choice-dynamic-programming-models-by-simulation-and-interpolation-monte-carlo-evidence). *Federal Reserve Bank of Minneapolis*, No. 181.